# Using Multiple Models

This notebook contains code to do the following:

* Convert Python code to C++

* Add docstrings or comments to Python code
  
* Create Unit tests

* Buy/Sell Equatities

In [1]:
# imports
import os
import gradio as gr

from dotenv import load_dotenv
from IPython.display import Markdown

#import io
#import sys
#import json
#import requests

from openai import OpenAI
import google.generativeai
import anthropic
import ollama

#import subprocess

In [2]:
# environment
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
# initialize LLMs
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"
GEMINI_MODEL = "gemini-1.5-flash"
OLLAMA_MODEL = "llama3.2"

In [4]:
# Prompt Class
class MessageHandler:
    def __init__(self, system_message, user_prompt, python_code):
        self.system_message = system_message
        self.user_prompt = user_prompt
        self.user_prompt_for = user_prompt + "\n" + python_code
        self.messages_for = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": self.user_prompt_for}
        ]

In [5]:
# Construct Prompt
def get_prompt(python_code, feature):
    system_message = ""
    user_prompt = ""
    if feature=="Code Conversion":
        system_message = "You are an assistant that reimplements Python code in high performance C++ for an M4 Mac. "
        system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
        system_message += "The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."
        
        user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
        user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
        user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    
    elif feature=="Add Comments":
        system_message = "You are an assistant specialized in adding clear and concise docstrings and comments to Python code to enhance its readability and maintainability. "
        system_message += "Respond exclusively using valid Python comments and docstrings."
                
        user_prompt = "Add appropriate comments and docstrings to the Python code provided below. Do not modify the code itself. "
        user_prompt += "Avoid summarizing or providing notes on your additions. Place comments for parameters outside the parentheses. "
        user_prompt += "Any output occurring before the first import statement should be enclosed within comments. Do not output the word 'python' at the top."
        user_prompt += "Return the modified code with the added explanations and documentation.\n\n"
    
    elif feature=="Generate Unit Tests":
        system_message = "You are an assistant specialized in generating unit tests for Python code to ensure its accuracy and reliability. "
        system_message += "Respond only with valid Python code."
                
        user_prompt = "Create unit tests for the Python code provided below. "
        user_prompt += "Return only the unit test code. Do not output the word 'python' at the top.\n\n"
    
    elif feature=="Generate Trading Bot Code":
        system_message = "You are a Python developer specializing in creating trading bots for buying and selling equities. "
        system_message += "Write accurate and functional Python code for executing equity trades."
                
        user_prompt = "Write Python code to buy and sell equities using the Alpaca API. "
        user_prompt += "Provide only the Python code as the output. Do not output the word 'python' at the top.\n\n"
    
    else:
        raise ValueError("Unknown Use Case")

    prompt = MessageHandler(system_message, user_prompt, python_code)
    return prompt

## Streaming output with different LLMs

In [6]:
def stream_gpt(prompt):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=prompt.messages_for, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

def stream_claude(prompt):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=prompt.system_message,
        messages=[{"role": "user", "content": prompt.user_prompt_for}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

def stream_gemini(prompt):
    gemini = google.generativeai.GenerativeModel(
        model_name=GEMINI_MODEL,
        system_instruction=prompt.system_message
    )
    
    response = gemini.generate_content(
        prompt.user_prompt_for,
        stream=True
    )

    reply = ""
    # Process the stream
    for chunk in response:
        # Extract text from the chunk
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```cpp\n','').replace('```','')

def stream_Llama(prompt):
    stream = ollama.chat(
        model=OLLAMA_MODEL,
        messages=prompt.messages_for,
        stream=True
    )

    reply = ""
    for chunk in stream:
        fragment = chunk["message"]["content"]
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

# Main

In [7]:
def get_results(python, model, feature):
    if feature=="Code Conversion":
        prompt = get_prompt(python, feature)
    elif feature=="Add Comments":
        prompt = get_prompt(python, feature)
    elif feature=="Generate Unit Tests":
        prompt = get_prompt(python, feature)    
    elif feature=="Generate Trading Bot Code":
        prompt = get_prompt(python, feature) 
    else:
        raise ValueError("Unknown Use Case")

    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    elif model=="Llama":
        result = stream_Llama(prompt)
    else:
        raise ValueError("Unknown Model")
    
    for stream_so_far in result:
        yield stream_so_far   

In [8]:
css = """
.python {background-color: #ADD8E6;}
.output {background-color: #fefbd9;}
"""

def read_input(user_input):
    return f"You entered: {user_input}"

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Multi-model Multi-features Exercise")
    with gr.Row():
        with gr.Column():
            fn = read_input,
            python = gr.Textbox(label="Enter Python Code Here:", lines=10, elem_classes=["python"])
        with gr.Column():
            with gr.Row():
                model = gr.Dropdown(["GPT", "Claude", "Gemini", "Llama"], label="Select model", value="GPT")
            with gr.Row():
                feature = gr.Dropdown(["Code Conversion", "Add Comments", "Generate Unit Tests", "Generate Trading Bot Code"], label="Select Use Case", value="Code Conversion") 
        with gr.Column():
            output = gr.Textbox(label="Output:", lines=10, elem_classes=["output"])
    with gr.Row():
        run = gr.Button("Run")
    
    run.click(get_results, inputs=[python, model, feature], outputs=[output])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
